# DX 799 Week 2 



In [ ]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

CSV = "COP.csv"
TARGET_YEAR = "2023"  

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row_idx = None
for i, row in raw.iterrows():
    year_like = sum(str(x).strip().isdigit() and 1990 <= int(x) <= 2025 for x in row)
    if year_like >= 5:
        header_row_idx = i
        break
if header_row_idx is None:
    raise ValueError("Could not locate a header row with year columns.")

df = pd.read_csv(CSV, encoding="latin1", header=header_row_idx)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = [c for c in df.columns if c.isdigit() and 1990 <= int(c) <= 2025]
if not year_cols:
    raise ValueError("No year columns found after parsing.")

if TARGET_YEAR not in df.columns:
    raise ValueError(f"Column '{TARGET_YEAR}' not found in the file.")

feature_years = [y for y in year_cols if int(y) < int(TARGET_YEAR)]
if not feature_years:
    raise ValueError("No feature years found before the target year.")

def to_num_frame(frame):
    out = frame.astype(str).str.replace(",", "", regex=False)
    out = pd.to_numeric(out, errors="coerce")
    return out

Xraw = df[feature_years].apply(to_num_frame, axis=0)
yraw = to_num_frame(df[TARGET_YEAR])

mask = yraw.notna()
X = Xraw.loc[mask].copy()
y = yraw.loc[mask].copy()
X = X.fillna(X.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

model = Pipeline([
    ("scale", StandardScaler()),
    ("lasso", Lasso(alpha=0.1, max_iter=10000, random_state=42))
])

model.fit(X_train, y_train)

print("Lasso R² score:", round(model.score(X_test, y_test), 4))
coef = model.named_steps["lasso"].coef_
print("Non-zero coefficients:", int((coef != 0).sum()), "/", len(coef))
print("Coefficients by year:")
for yr, c in zip(X.columns, coef):
    print(yr, ":", round(c, 6))


Lasso R² score: 0.9996
Non-zero coefficients: 8 / 33
Coefficients by year:
1990 : -0.0
1991 : -15.215393
1992 : -0.0
1993 : -0.0
1994 : -0.0
1995 : -0.0
1996 : 0.0
1997 : 0.0
1998 : 0.0
1999 : 0.0
2000 : 0.0
2001 : 16.788625
2002 : 29.211767
2003 : -0.0
2004 : -0.0
2005 : -0.0
2006 : -0.0
2007 : -0.0
2008 : -151.608427
2009 : -0.0
2010 : -0.0
2011 : -0.0
2012 : -0.0
2013 : 0.0
2014 : 177.036899
2015 : 0.0
2016 : -0.0
2017 : 0.0
2018 : 0.0
2019 : 0.0
2020 : 215.533431
2021 : 64.045534
2022 : 93.86648


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

CSV = "EP.csv"

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row = None
for i, row in raw.iterrows():
    year_like = sum(str(x).strip().isdigit() and 1990 <= int(x) <= 2100 for x in row)
    if year_like >= 5:
        header_row = i
        break
if header_row is None:
    raise ValueError("Couldn't find a header row with year columns in EP.csv")

df = pd.read_csv(CSV, encoding="latin1", header=header_row)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = sorted([c for c in df.columns if c.isdigit() and 1900 <= int(c) <= 2100],
                   key=lambda x: int(x))
if len(year_cols) < 2:
    raise ValueError("Not enough year columns detected.")

target_year = year_cols[-1]
feature_years = year_cols[:-1]

def clean_numeric(df_subset):
    return df_subset.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(",", "", regex=False),
                                                   errors="coerce"))

X = clean_numeric(df[feature_years])
y = pd.to_numeric(df[target_year].astype(str).str.replace(",", "", regex=False), errors="coerce")

mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.fillna(X.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Pipeline([
    ("scale", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=42))
])
model.fit(X_train, y_train)

print(f"Target year: {target_year}")
print("Ridge R² score:", round(model.score(X_test, y_test), 4))
print("Coefficients by year:")
for year, coef in zip(feature_years, model.named_steps["ridge"].coef_):
    print(year, ":", round(coef, 6))


Target year: 2023
Ridge R² score: 0.9998
Coefficients by year:
1990 : -57.26556
1991 : -61.860838
1992 : -66.247132
1993 : -55.885776
1994 : -78.18878
1995 : -85.312752
1996 : -95.127933
1997 : -102.184914
1998 : -100.033512
1999 : -97.292106
2000 : -97.643916
2001 : -120.894357
2002 : -91.210055
2003 : -67.841299
2004 : -44.851583
2005 : -14.379291
2006 : 22.513803
2007 : 59.432749
2008 : 75.058445
2009 : 126.161048
2010 : 146.385195
2011 : 203.139139
2012 : 221.312546
2013 : 266.359544
2014 : 314.241927
2015 : 301.351744
2016 : 318.083935
2017 : 338.12063
2018 : 384.967377
2019 : 416.348521
2020 : 446.726158
2021 : 468.625402
2022 : 511.589553


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso

CSV = "TED.csv"

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row = None
for i, row in raw.iterrows():
    year_like = sum(str(x).strip().isdigit() and 1900 <= int(x) <= 2100 for x in row)
    if year_like >= 5:   
        header_row = i
        break
if header_row is None:
    raise ValueError("Couldn't find a header row with year columns in TED.csv")

df = pd.read_csv(CSV, encoding="latin1", header=header_row)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = sorted([c for c in df.columns if c.isdigit() and 1900 <= int(c) <= 2100],
                   key=lambda x: int(x))
if len(year_cols) < 2:
    raise ValueError("Not enough year columns detected in TED.csv")
target_year = year_cols[-1]
feature_years = year_cols[:-1]

def clean_numeric(df_subset):
    return df_subset.apply(lambda col: pd.to_numeric(col.astype(str)
                                                    .str.replace(",", "", regex=False),
                                                    errors="coerce"))

X = clean_numeric(df[feature_years])
y = pd.to_numeric(df[target_year].astype(str).str.replace(",", "", regex=False), errors="coerce")

mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.fillna(X.median(numeric_only=True))

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = Pipeline([("scale", StandardScaler()),
                  ("lasso", Lasso(alpha=0.1, max_iter=10000, random_state=42))])
model.fit(Xtr, ytr)

print(f"Target year: {target_year}")
print("Lasso R² score:", round(model.score(Xte, yte), 4))
coef = model.named_steps["lasso"].coef_
print("Non-zero coefficients:", int((coef != 0).sum()), "/", len(coef))
print("Coefficients by year:")
for yr, c in zip(feature_years, coef):
    print(f"{yr}: {round(c, 6)}")


Target year: 2023
Lasso R² score: 0.9999
Non-zero coefficients: 13 / 33
Coefficients by year:
1990: -81.219397
1991: -0.0
1992: -39.751677
1993: -66.391554
1994: 0.0
1995: 0.0
1996: 0.0
1997: 304.307519
1998: 6.424095
1999: 0.0
2000: 0.0
2001: 0.0
2002: 0.0
2003: -0.0
2004: -132.397432
2005: -221.70265
2006: -0.0
2007: -0.0
2008: -0.0
2009: 0.0
2010: -0.0
2011: -0.0
2012: -0.0
2013: -0.0
2014: 30.938343
2015: 0.0
2016: -93.233638
2017: -0.0
2018: -0.0
2019: 312.361471
2020: 678.846893
2021: 165.667558
2022: 642.704622


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

CSV = "TED.csv"

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row = None
for i, row in raw.iterrows():
    cnt = sum(str(v).strip().isdigit() and 1900 <= int(v) <= 2100 for v in row)
    if cnt >= 5:  
        header_row = i
        break
if header_row is None:
    raise ValueError("Couldn't find a header row with year columns in TED.csv")

df = pd.read_csv(CSV, encoding="latin1", header=header_row)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = sorted([c for c in df.columns if c.isdigit() and 1900 <= int(c) <= 2100],
                   key=lambda x: int(x))
if len(year_cols) < 2:
    raise ValueError("Not enough year columns detected in TED.csv")

target_year = year_cols[-1]
feature_years = year_cols[:-1]

def clean_numeric(df_subset):
    return df_subset.apply(lambda col: pd.to_numeric(col.astype(str)
                                                    .str.replace(",", "", regex=False),
                                                    errors="coerce"))
X = clean_numeric(df[feature_years])
y = pd.to_numeric(df[target_year].astype(str).str.replace(",", "", regex=False),
                  errors="coerce")

mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.fillna(X.median(numeric_only=True))

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = Pipeline([("scale", StandardScaler()),
                  ("ridge", Ridge(alpha=1.0, random_state=42))])
model.fit(Xtr, ytr)

print(f"Target year: {target_year}")
print("Ridge R² score:", round(model.score(Xte, yte), 4))
print("Coefficients by year:")
for yr, coef in zip(feature_years, model.named_steps["ridge"].coef_):
    print(f"{yr}: {round(coef, 6)}")


Target year: 2023
Ridge R² score: 0.9985
Coefficients by year:
1990: -64.159783
1991: -49.892858
1992: -33.85952
1993: -39.493492
1994: 16.075446
1995: 24.485741
1996: 27.807288
1997: 29.90976
1998: 21.466751
1999: 11.364918
2000: -9.931301
2001: -2.557804
2002: -7.736788
2003: -33.109196
2004: -32.918967
2005: -32.154446
2006: -15.535452
2007: 3.470445
2008: 7.748742
2009: 44.52639
2010: 41.675859
2011: 56.71878
2012: 66.719907
2013: 89.038614
2014: 136.698624
2015: 128.11308
2016: 69.913114
2017: 96.673768
2018: 145.035586
2019: 193.147287
2020: 187.848952
2021: 196.251903
2022: 222.529629


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

CSV = "EP.csv"

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row = None
for i, row in raw.iterrows():
    yearish = sum(str(v).strip().isdigit() and 1900 <= int(v) <= 2100 for v in row)
    if yearish >= 5:
        header_row = i
        break
if header_row is None:
    raise ValueError("Couldn't find a header row with year columns in EP.csv")

df = pd.read_csv(CSV, encoding="latin1", header=header_row)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = sorted([c for c in df.columns if c.isdigit() and 1900 <= int(c) <= 2100],
                   key=lambda x: int(x))
if len(year_cols) < 2:
    raise ValueError("Not enough year columns detected in EP.csv")

target_year = year_cols[-1]
feature_years = year_cols[:-1]

def clean_numeric(df_subset):
    return df_subset.apply(
        lambda col: pd.to_numeric(col.astype(str).str.replace(",", "", regex=False), errors="coerce")
    )

X = clean_numeric(df[feature_years])
y = pd.to_numeric(df[target_year].astype(str).str.replace(",", "", regex=False), errors="coerce")

mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.fillna(X.median(numeric_only=True))

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = Pipeline([("scale", StandardScaler()), ("ridge", Ridge(alpha=1.0, random_state=42))])
model.fit(Xtr, ytr)

print(f"Target year: {target_year}")
print("Ridge R² score:", round(model.score(Xte, yte), 4))
print("Coefficients by year:")
for yr, coef in zip(feature_years, model.named_steps["ridge"].coef_):
    print(f"{yr}: {round(coef, 6)}")


Target year: 2023
Ridge R² score: 0.9998
Coefficients by year:
1990: -57.26556
1991: -61.860838
1992: -66.247132
1993: -55.885776
1994: -78.18878
1995: -85.312752
1996: -95.127933
1997: -102.184914
1998: -100.033512
1999: -97.292106
2000: -97.643916
2001: -120.894357
2002: -91.210055
2003: -67.841299
2004: -44.851583
2005: -14.379291
2006: 22.513803
2007: 59.432749
2008: 75.058445
2009: 126.161048
2010: 146.385195
2011: 203.139139
2012: 221.312546
2013: 266.359544
2014: 314.241927
2015: 301.351744
2016: 318.083935
2017: 338.12063
2018: 384.967377
2019: 416.348521
2020: 446.726158
2021: 468.625402
2022: 511.589553


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge

CSV = "COP.csv"

raw = pd.read_csv(CSV, encoding="latin1", header=None)
header_row = None
for i, row in raw.iterrows():
    cnt = sum(str(v).strip().isdigit() and 1900 <= int(v) <= 2100 for v in row)
    if cnt >= 5:     # looks like a row of years
        header_row = i
        break
if header_row is None:
    raise ValueError("Couldn't find a header row with year columns in COP.csv")

df = pd.read_csv(CSV, encoding="latin1", header=header_row)
df = df.dropna(axis=1, how="all")
df.columns = df.columns.astype(str).str.strip()
df = df.rename(columns={df.columns[0]: "Country"})

year_cols = sorted([c for c in df.columns if c.isdigit() and 1900 <= int(c) <= 2100],
                   key=lambda x: int(x))
if len(year_cols) < 2:
    raise ValueError("Not enough year columns detected in COP.csv")

target_year = year_cols[-1]
feature_years = year_cols[:-1]

def clean_numeric(df_subset):
    return df_subset.apply(
        lambda col: pd.to_numeric(col.astype(str).str.replace(",", "", regex=False), errors="coerce")
    )

X = clean_numeric(df[feature_years])
y = pd.to_numeric(df[target_year].astype(str).str.replace(",", "", regex=False), errors="coerce")

mask = y.notna()
X, y = X.loc[mask], y.loc[mask]
X = X.fillna(X.median(numeric_only=True))

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.2, random_state=42)
model = Pipeline([("scale", StandardScaler()), ("ridge", Ridge(alpha=1.0, random_state=42))])
model.fit(Xtr, ytr)

print(f"Target year: {target_year}")
print("Ridge R² score:", round(model.score(Xte, yte), 4))
print("Coefficients by year:")
for yr, coef in zip(feature_years, model.named_steps["ridge"].coef_):
    print(f"{yr}: {round(coef, 6)}")


Target year: 2023
Ridge R² score: 0.9988
Coefficients by year:
1990: 11.871785
1991: 11.125447
1992: 6.990966
1993: 4.652049
1994: 4.242378
1995: 6.977675
1996: 12.296092
1997: 13.617239
1998: 5.444526
1999: 2.185372
2000: -1.739768
2001: 5.49368
2002: 10.745134
2003: -10.836755
2004: -17.781809
2005: -24.779815
2006: -21.954902
2007: -20.501239
2008: -30.423416
2009: -10.049078
2010: -8.132164
2011: -16.14791
2012: -2.824977
2013: 18.181262
2014: 41.861393
2015: 42.711247
2016: 17.2157
2017: 31.944421
2018: 46.954452
2019: 71.406717
2020: 77.155338
2021: 79.938685
2022: 71.825724
